In [1]:
import numpy as np
import pandas as pd
import torch
import tensorly as tl
# library for TTD   (Tensorly  (search))

from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tensorly.decomposition import tensor_train
from tensorly.decomposition import tensor_train
# different forms of tensor
from sklearn.preprocessing import MinMaxScaler
from tensorly import tt_to_tensor
from tensorly.decomposition import matrix_product_state
# tensor operations


#from math import ceil
#from google.colab import drive
#drive.mount('/content/gdrive')
import torch.nn.init as init
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader
import time
import os
import copy
import torch.nn.functional as Fun
print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
print("GPU is available?", torch.cuda.is_available())

PyTorch Version: 1.13.1
Torchvision Version: 0.14.1
GPU is available? True


In [2]:
dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
features = list()
with open("/home/c/cl237/TenBCD/UCI HAR/data/features.txt") as f:
    features = [line.split()[1] for line in f.readlines()]
print('# of Features: {}'.format(len(features)))

# of Features: 561


## get train data

In [4]:
pd_X_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_train.txt', delim_whitespace=True, header=None)
pd_X_train.columns = features
pd_y_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_train.txt', delim_whitespace=True, names=['Activity'])

N = len(pd_X_train)

K = 6

x = pd_X_train.values
y = pd_y_train.values

X_train=torch.tensor(x,dtype = torch.float32,device=device)
y_train=torch.tensor(y,dtype = int) #dtype=torch.int,device=device)

y_train = torch.flatten(y_train)

X_train = torch.t(X_train)

y_one_hot = torch.zeros(N, K).scatter_(1, torch.reshape(y_train-1, (N, 1)), 1)
y_one_hot = torch.t(y_one_hot).to(device=device)
y_train = y_train.to(device=device)

## get test data

In [5]:
pd_X_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_test.txt', delim_whitespace=True, header=None)
pd_X_test.columns = features
pd_y_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_test.txt', delim_whitespace=True, names=['Activity'])

N_test = len(pd_X_test)

x = pd_X_test.values
y = pd_y_test.values

X_test=torch.tensor(x,dtype = torch.float32, device=device)
y_test=torch.tensor(y,dtype = int)

y_test = torch.flatten(y_test)

X_test = torch.t(X_test)

y_test_one_hot = torch.zeros(N_test, K).scatter_(1, torch.reshape(y_test-1, (N_test, 1)), 1)
y_test_one_hot = torch.t(y_test_one_hot).to(device=device)
y_test = y_test.to(device=device)

In [6]:
def updateV(U1,U2,W,b,rho,gamma): 
    _, d = W.size()
    I = torch.eye(d, device=device) #Returns a 2-D tensor with ones on the diagonal and zeros elsewhere.
    U1 = nn.ReLU()(U1)
    _, col_U2 = U2.size()
    Vstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(W),W))+gamma*I), rho*torch.mm(torch.t(W),U2-b.repeat(1,col_U2))+gamma*U1)
    # check how to get V* W*
    # torch.mm*=(input,mat2) performs a matrix multiplication of the matrices input and mat2.
    #   if input is a (n*m) tensor,mat2 is a (m*p) tensor, output will be (n*p) tensor
    return Vstar

def updateWb_org(U, V, W, b, alpha, rho): 
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse(alpha*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

def updateWb(U, V, W, b, W_tensor_rec, alpha, rho,tau): 
    W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
    W_tensor2matrix = W_tensor_rec.reshape(W.shape)
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+tau*W_tensor2matrix+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse((alpha+tau)*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

# U update  E(1) in global convergent
def relu_prox(a, b, gamma, d, N):
    val = torch.empty(d,N, device=device)
    x = (a+gamma*b)/(1+gamma)
    y = torch.min(b,torch.zeros(d,N, device=device))

    val = torch.where(a+gamma*b < 0, y, torch.zeros(d,N, device=device))
    val = torch.where(((a+gamma*b >= 0) & (b >=0)) | ((a*(gamma-np.sqrt(gamma*(gamma+1))) <= gamma*b) & (b < 0)), x, val)
    val = torch.where((-a <= gamma*b) & (gamma*b <= a*(gamma-np.sqrt(gamma*(gamma+1)))), b, val)
    return val

In [8]:
########### parameter setup
#df = pd.DataFrame()
#df.to_csv('C:/Users/Mark/Desktop/rank40_100times.csv')
niter = 500
rank = 80
tau = 5
alpha = 1
gamma = 0.5
rho = 0.5

#alpha1 = alpha2 = alpha3 = alpha4 = alpha5 = alpha6 = alpha7 = alpha8 = alpha9 = alpha10 = alpha
#for rank in (35,): #(25,30,35)(100,180,220,260,300,340,380)(20,60,100,140,180,220,260,300,340,380)
#    for tau in (400,500): #(300,400,500)(10,50,100,200,300)(10,50,100,200,300)
#        for gamma in (0.5,0.8,2): #(0.5,0.8,2)(0.5,0.8)(0.5,1,1.5,2,3)
            #gamma1 = gamma2 = gamma3 = gamma4 = gamma5 = gamma
#            for rho in (0.5,0.8,2): #(0.5,0.8)(1,2)
                #rho1 = rho2 = rho3 = rho4 = rho5= rho
#                for alpha in (0.5,1,1.5,2):
#                    print ("rank=",rank, "tau=",tau, "gamma=",gamma, "rho=",rho, "alpha",alpha)
                    #print('Compression Ratio', ((1024*28*28+10*1024+(8*(rank)+32*np.square(rank))*2)/(1024*28*28+10*1024+1024*1024*2)), (8*(rank)+32*np.square(rank))*2/(1024*1024*2))
        
print ("rank=",rank, "tau=",tau, "gamma=",gamma, "rho=",rho, "alpha",alpha)

loss1 = np.empty(niter)
loss2 = np.empty(niter)
accuracy_train = np.empty(niter)
accuracy_test = np.empty(niter)
time1 = np.empty(niter)

results = torch.zeros(1, 5, niter)



for Out_iter in range(1):
    rank_initial = 400
    seed = 10 + 10*Out_iter
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)

    d0 = 561 #561 =3*11*17
    d1 =  1024    # 2^10
    d2 =  1024  
    d3 = 1024
    d4 = 512      # 2^9
    d5 = 6 

    limit = torch.sqrt(torch.tensor(3. / d0))
    W1 = 0.2* torch.empty(d1, d0, device=device).uniform_(-limit, limit)
    W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
    W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
    factors1 = tensor_train(W1_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W1_tl_tensor_rec = tt_to_tensor(factors1)
    b1 = 0*torch.ones(d1, 1, device=device) # 0 is stable


    limit = torch.sqrt(torch.tensor(3. / d1))
    W2 = 0.2* torch.empty(d2, d1, device=device).uniform_(-limit, limit)
    W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy())
    factors2 = tensor_train(W2_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    # we have 9 rank_initial (r1,...,r9) here plus 2 of 1s (r0 & r10).
    # factors: set of tensor cores
    # http://tensorly.org/stable/user_guide/tensor_decomposition.html
    # http://tensorly.org/stable/modules/generated/tensorly.decomposition.tensor_train.html#tensorly.decomposition.tensor_train
    W2_tl_tensor_rec = tt_to_tensor(factors2)
    # tt_to_tensor(factors):Re-assembles ‘factors’, which represent a tensor in TT format into the corresponding full tensor
    #      facros: list of 3d-arrays tt-cores           output_tensor: ndarray      
    b2 = 0*torch.ones(d2, 1, device=device)


    limit = torch.sqrt(torch.tensor(3. / d2))
    W3 = 0.2* torch.empty(d3, d2, device=device).uniform_(-limit, limit)
    W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
    factors3 = tensor_train(W3_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W3_tl_tensor_rec = tt_to_tensor(factors3)
    b3 = 0*torch.ones(d3, 1, device=device)

    limit = torch.sqrt(torch.tensor(3. / d3))
    W4 = 0.2* torch.empty(d4, d3, device=device).uniform_(-limit, limit)
    W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2)) # 8 number of 4s, 2 number of 8s
    W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    factors4 = tensor_train(W4_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W4_tl_tensor_rec = tt_to_tensor(factors4)
    b4 = 0*torch.ones(d4, 1, device=device)


    limit = torch.sqrt(torch.tensor(3. / d4))
    W5 = 0.2* torch.empty(d5, d4, device=device).uniform_(-limit, limit)
    b5 = 0*torch.ones(d5, 1, device=device)

    # W4 = 0.01*torch.randn(d4, d3, device=device)
    # W4_torch_tensor = W4.reshape((50,30,K))
    # W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    # core4, tucker_factors4 = tucker(W4_tl_tensor, rank=(40,28,K), init='svd', tol=10e-5, random_state=12345)
    # W4_tl_tensor_rec = tl.tucker_to_tensor((core4, tucker_factors4))
    # b4 = 0.1*torch.ones(d4, 1, device=device)

    U1 = torch.addmm(b1.repeat(1, N), W1, X_train)
    V1 = nn.ReLU()(U1)
    U2 = torch.addmm(b2.repeat(1, N), W2, V1)
    V2 = nn.ReLU()(U2)
    U3 = torch.addmm(b3.repeat(1, N), W3, V2)
    V3 = nn.ReLU()(U3)
    U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    V4 = nn.ReLU()(U4)
    U5 = torch.addmm(b5.repeat(1, N), W5, V4)
    V5 = U5 
    # U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    # V4 = U4



    # Iterations
    print('Train on', N, 'samples, validate on', N_test, 'samples')
    for k in range(niter):
        start = time.time()

 # update for last layer
        # update V4
        V5 = (y_one_hot + gamma*U5 + alpha*V5)/(1 + gamma + alpha)

        # update U4 
        U5 = (gamma*V5 + rho*(torch.mm(W5,V4) + b5.repeat(1,N)))/(gamma + rho)

        # update W4 and b4
        W5, b5 = updateWb_org(U5,V4,W5,b5,alpha,rho)
        # W3_torch_tensor = W3.reshape((d3,4,4,4,4,4,4))
        # W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
        # factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,1))
        # W3_tl_tensor_rec = tt_to_tensor(factors3)



  # update for 4th layer
        # update V3
        V4 = updateV(U4,U5,W5,b5,rho,gamma)

        # update U3
        U4 = relu_prox(V4,(rho*torch.addmm(b4.repeat(1,N), W4, V3) + alpha*U4)/(rho + alpha),(rho + alpha)/gamma,d4,N)

        # update W3 and b3
        W4, b4 = updateWb(U4,V3,W4,b4,W4_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2))
        W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors4 = tensor_train(W4_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W4_tl_tensor_rec = tt_to_tensor(factors4)

  # update for 3nd layer
        # update V3
        V3 = updateV(U3,U4,W4,b4,rho,gamma)

        # update U3
        U3 = relu_prox(V3,(rho*torch.addmm(b3.repeat(1,N), W3, V2) + alpha*U3)/(rho + alpha),(rho + alpha)/gamma,d3,N)

        # update W3 and b3
        W3, b3 = updateWb(U3,V2,W3,b3,W3_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4))
        W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W3_tl_tensor_rec = tt_to_tensor(factors3)

 # update for 2nd layer
        # update V2
        V2 = updateV(U2,U3,W3,b3,rho,gamma)

        # update U2
        U2 = relu_prox(V2,(rho*torch.addmm(b2.repeat(1,N), W2, V1) + alpha*U2)/(rho + alpha),(rho + alpha)/gamma,d2,N)

        # update W2 and b2
        W2, b2 = updateWb(U2,V1,W2,b2,W2_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4))
        W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy()) 
        factors2 = tensor_train(W2_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W2_tl_tensor_rec = tt_to_tensor(factors2)


# update for 1st layer
        # update V1
        V1 = updateV(U1,U2,W2,b2,rho,gamma)

        # update U1
        U1 = relu_prox(V1,(rho*torch.addmm(b1.repeat(1,N), W1, X_train) + alpha*U1)/(rho + alpha),(rho + alpha)/gamma,d1,N)

        # update W1 and b1
        W1, b1 = updateWb(U1,X_train,W1,b1,W1_tl_tensor_rec, alpha,rho,tau)

        # G update
        W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
        W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
        factors1 = tensor_train(W1_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W1_tl_tensor_rec = tt_to_tensor(factors1)




 # prediction for trainning data
        a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_train))
        #a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train))
        a4_train = nn.ReLU()(torch.addmm(b4.repeat(1, N), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_train))
        #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        pred = torch.argmax(torch.addmm(b5.repeat(1, N), W5, a4_train), dim=0)
        # check argmax and addmm, dim=0

 #Prediction for test data
        a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_test))
        #a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test))
        a4_test = nn.ReLU()(torch.addmm(b4.repeat(1, N_test), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_test))
        pred_test = torch.argmax(torch.addmm(b5.repeat(1, N_test), W5, a4_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        # # a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_train))
        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        # # a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), W2, a1_train))

        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), W3, a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), W2, a1_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), W3, a2_test), dim=0)




   #emperical loss
        loss1[k] = gamma/2*torch.pow(torch.dist(V5,y_one_hot,2),2).cpu().numpy()
        # torch.pow:Takes the power of each element in input with exponent and returns a tensor with the result.
        # torch.disk: Returns the p-norm of (input - other)
        # Eq (5) in paper
        loss2[k] = loss1[k] + rho/2*torch.pow(torch.dist(torch.addmm(b1.repeat(1,N), W1, X_train),U1,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b2.repeat(1,N), W2, V1),U2,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b3.repeat(1,N), W3, V2),U3,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b4.repeat(1,N), W4, V3),U4,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b5.repeat(1,N), W5, V4),U5,2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V1,nn.ReLU()(U1),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V2,nn.ReLU()(U2),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V3,nn.ReLU()(U3),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V4,nn.ReLU()(U4),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V5,U5,2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W1.reshape((6,22,34,2,2,2,2,2,2,2)),torch.as_tensor(W1_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W2.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W2_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W3.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W4.reshape((4,4,4,4,4,4,4,4,4,2)),torch.as_tensor(W4_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \

        # +tau/2*torch.pow(torch.dist(W3.reshape((d3,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() 
        # +tau/2*torch.pow(torch.dist(W4.reshape((50,30,K)),torch.as_tensor(W4_tl_tensor_rec,device=device),2),2).cpu().numpy() 

        # compute training accuracy
        correct_train = pred == y_train-1
        accuracy_train[k] = np.mean(correct_train.cpu().numpy())

        # compute validation accuracy
        correct_test = pred_test == y_test-1
        accuracy_test[k] = np.mean(correct_test.cpu().numpy())

        # compute training time
        stop = time.time()
        duration = stop - start
        time1[k] = duration

        # print results
        print('Repeatition', Out_iter + 1, 'Epoch', k + 1, '/', niter, '\n', 
              '-', 'time:', time1[k], '-', 'sq_loss:', loss1[k], '-', 'tot_loss:', loss2[k], 
              '-', 'acc:', accuracy_train[k], '-', 'val_acc:', accuracy_test[k])

##############
############## compute CR
    factors1_shape=[f.shape for f in factors1]
    Sum_of_variables_factors1=sum(list(x*y*z for x,y,z in factors1_shape))
    factors2_shape=[f.shape for f in factors2]
    Sum_of_variables_factors2=sum(list(x*y*z for x,y,z in factors2_shape))
    factors3_shape=[f.shape for f in factors3]
    Sum_of_variables_factors3=sum(list(x*y*z for x,y,z in factors3_shape))
    factors4_shape=[f.shape for f in factors4]
    Sum_of_variables_factors4=sum(list(x*y*z for x,y,z in factors4_shape))

    total_variabels=Sum_of_variables_factors1+Sum_of_variables_factors2+Sum_of_variables_factors3+Sum_of_variables_factors4

    CR_1=((total_variabels)+(d4*d5))/(d0*d1+d1*d2+d2*d3+d3*d4+d4*d5)
    CR_2=(total_variabels)/(d0*d1+d1*d2+d2*d3+d3*d4)
    print("CR_1 = "+str(CR_1)+"   "+"CR_2 = " +str(CR_2))


    results[Out_iter,0,:] = torch.tensor(loss1)
    results[Out_iter,1,:] = torch.tensor(loss2)
    results[Out_iter,2,:] = torch.tensor(accuracy_train)
    results[Out_iter,3,:] = torch.tensor(accuracy_test)
    results[Out_iter,4,:] = torch.tensor(time1)
    CR=(CR_1,CR_2)

#this postion to add new row into existing table
    #df=pd.read_csv('C:/Users/Mark/Desktop/rank40_100times.csv')
    #new_row = {'rank':rank, 'CR_1':CR_1, 'CR_2':CR_2, 'tau':tau, 'gamma':gamma, 'rho':rho, 'alpha':alpha, \
    #           'loss1':loss1[niter-1], 'loss2':loss2[niter-1], 'accuracy_train':accuracy_train[niter-1], \
    #           'accuracy_test':accuracy_test[niter-1], 'time':time1[niter-1], 'seed':seed} 
    #df=df.append(new_row,ignore_index=True)
    #df.to_csv('C:/Users/Mark/Desktop/rank40_100times.csv',index=False)


filename= "LecunUniform_" + "niter_"+ str(niter) + "rank_" + str(rank) + "tau_" + str(tau) + "gamma_" + str(gamma) + \
"rho_" + str(rho) + "alpha_" + str(alpha) + ".mat"
from scipy.io import savemat
%cd /home/c/cl237/TenBCD/UCI HAR/4 hidden layers/Different Initialization/
savemat (filename, {'results': torch.Tensor.numpy(results), 'Compression Ratio':CR})
#this position to save table into matlab
#df.to_csv('C:/Users/Mark/Desktop/result_compress_123_layer_highrank.csv')
##Changing Folder
 #%cd '/content/gdrive/MyDrive/Colab/Tensor-BCD-for-DNN/Experiments/DataSaved'
 #savemat("ThreeLayer_rank160.mat", {'results': torch.Tensor.numpy(results)})    

rank= 80 tau= 5 gamma= 0.5 rho= 0.5 alpha 1
Train on 7352 samples, validate on 2947 samples
Repeatition 1 Epoch 1 / 500 
 - time: 1.2714040279388428 - sq_loss: 661.6912231445312 - tot_loss: 1029.325135433326 - acc: 0.1838955386289445 - val_acc: 0.1818798778418731
Repeatition 1 Epoch 2 / 500 
 - time: 1.236530065536499 - sq_loss: 294.0849914550781 - tot_loss: 486.38769765850157 - acc: 0.1868879216539717 - val_acc: 0.18052256532066507
Repeatition 1 Epoch 3 / 500 
 - time: 1.2426419258117676 - sq_loss: 161.3899383544922 - tot_loss: 268.4336228463799 - acc: 0.1868879216539717 - val_acc: 0.18052256532066507
Repeatition 1 Epoch 4 / 500 
 - time: 1.2341430187225342 - sq_loss: 87.46602630615234 - tot_loss: 152.6410679658875 - acc: 0.1868879216539717 - val_acc: 0.18052256532066507
Repeatition 1 Epoch 5 / 500 
 - time: 1.238257884979248 - sq_loss: 47.03624725341797 - tot_loss: 89.95820982242003 - acc: 0.1868879216539717 - val_acc: 0.18052256532066507
Repeatition 1 Epoch 6 / 500 
 - time: 1.23906

Repeatition 1 Epoch 48 / 500 
 - time: 1.2174012660980225 - sq_loss: 0.0004885353264398873 - tot_loss: 1.124499349425605 - acc: 0.8813928182807399 - val_acc: 0.8445877163216831
Repeatition 1 Epoch 49 / 500 
 - time: 1.2388477325439453 - sq_loss: 0.0004554734914563596 - tot_loss: 1.0722286274885846 - acc: 0.8853373231773667 - val_acc: 0.8479809976247031
Repeatition 1 Epoch 50 / 500 
 - time: 1.226099967956543 - sq_loss: 0.00042506863246671855 - tot_loss: 1.0269144911217154 - acc: 0.8881936887921654 - val_acc: 0.8537495758398371
Repeatition 1 Epoch 51 / 500 
 - time: 1.241323471069336 - sq_loss: 0.0003970947000198066 - tot_loss: 0.9805205897991982 - acc: 0.8915941240478781 - val_acc: 0.8557855446216491
Repeatition 1 Epoch 52 / 500 
 - time: 1.2379484176635742 - sq_loss: 0.00037121635978110135 - tot_loss: 0.9377796541903081 - acc: 0.8943144722524483 - val_acc: 0.8601968103155752
Repeatition 1 Epoch 53 / 500 
 - time: 1.2249748706817627 - sq_loss: 0.0003473201359156519 - tot_loss: 0.895828

Repeatition 1 Epoch 95 / 500 
 - time: 1.2397406101226807 - sq_loss: 3.826139072771184e-05 - tot_loss: 0.26666555748624887 - acc: 0.9491294885745375 - val_acc: 0.9277231082456736
Repeatition 1 Epoch 96 / 500 
 - time: 1.2470693588256836 - sq_loss: 3.6813704355154186e-05 - tot_loss: 0.25665707800203563 - acc: 0.9500816104461371 - val_acc: 0.9277231082456736
Repeatition 1 Epoch 97 / 500 
 - time: 1.2353193759918213 - sq_loss: 3.5444321838440374e-05 - tot_loss: 0.24612129483853096 - acc: 0.9511697497279652 - val_acc: 0.9284017645062775
Repeatition 1 Epoch 98 / 500 
 - time: 1.2609765529632568 - sq_loss: 3.41423146892339e-05 - tot_loss: 0.24187668465742718 - acc: 0.9528019586507073 - val_acc: 0.9294197488971836
Repeatition 1 Epoch 99 / 500 
 - time: 1.224395513534546 - sq_loss: 3.290054155513644e-05 - tot_loss: 0.24468824722453064 - acc: 0.9536180631120783 - val_acc: 0.9297590770274856
Repeatition 1 Epoch 100 / 500 
 - time: 1.2345430850982666 - sq_loss: 3.1730145565234125e-05 - tot_loss: 

Repeatition 1 Epoch 141 / 500 
 - time: 1.2342028617858887 - sq_loss: 1.2558539310703054e-05 - tot_loss: 0.13852332742287388 - acc: 0.9730685527747551 - val_acc: 0.9446895147607737
Repeatition 1 Epoch 142 / 500 
 - time: 1.233992099761963 - sq_loss: 1.2402019820001442e-05 - tot_loss: 0.14389325682957121 - acc: 0.9729325353645266 - val_acc: 0.9450288428910757
Repeatition 1 Epoch 143 / 500 
 - time: 1.2416298389434814 - sq_loss: 1.225904816237744e-05 - tot_loss: 0.1386993458895205 - acc: 0.9733405875952121 - val_acc: 0.9446895147607737
Repeatition 1 Epoch 144 / 500 
 - time: 1.255936861038208 - sq_loss: 1.2108343071304262e-05 - tot_loss: 0.13756050957100285 - acc: 0.9737486398258978 - val_acc: 0.9450288428910757
Repeatition 1 Epoch 145 / 500 
 - time: 1.2409954071044922 - sq_loss: 1.1958059076278005e-05 - tot_loss: 0.13160734503807703 - acc: 0.9742927094668118 - val_acc: 0.9450288428910757
Repeatition 1 Epoch 146 / 500 
 - time: 1.2400860786437988 - sq_loss: 1.1817162885563448e-05 - tot_

Repeatition 1 Epoch 187 / 500 
 - time: 1.2587344646453857 - sq_loss: 8.668454029248096e-06 - tot_loss: 0.10873190661378374 - acc: 0.9804134929270947 - val_acc: 0.9518154054971157
Repeatition 1 Epoch 188 / 500 
 - time: 1.2267069816589355 - sq_loss: 8.624885595054366e-06 - tot_loss: 0.11155291012283897 - acc: 0.9804134929270947 - val_acc: 0.9521547336274178
Repeatition 1 Epoch 189 / 500 
 - time: 1.2380762100219727 - sq_loss: 8.580527719459496e-06 - tot_loss: 0.11714074353147907 - acc: 0.9804134929270947 - val_acc: 0.9524940617577197
Repeatition 1 Epoch 190 / 500 
 - time: 1.236562728881836 - sq_loss: 8.540847375115845e-06 - tot_loss: 0.11046919929213317 - acc: 0.9806855277475517 - val_acc: 0.9524940617577197
Repeatition 1 Epoch 191 / 500 
 - time: 1.2335045337677002 - sq_loss: 8.501283446094021e-06 - tot_loss: 0.1118434271267219 - acc: 0.9808215451577802 - val_acc: 0.9524940617577197
Repeatition 1 Epoch 192 / 500 
 - time: 1.2338666915893555 - sq_loss: 8.459328455501236e-06 - tot_loss

...

Repeatition 1 Epoch 279 / 500 
 - time: 1.2534747123718262 - sq_loss: 6.2609474298369605e-06 - tot_loss: 0.08888278587353682 - acc: 0.9868063112078346 - val_acc: 0.9579233118425518
Repeatition 1 Epoch 280 / 500 
 - time: 1.236687183380127 - sq_loss: 6.243325060495408e-06 - tot_loss: 0.09555315125663455 - acc: 0.9866702937976061 - val_acc: 0.9572446555819477
Repeatition 1 Epoch 281 / 500 
 - time: 1.2305853366851807 - sq_loss: 6.227133326319745e-06 - tot_loss: 0.0938935235365399 - acc: 0.9866702937976061 - val_acc: 0.9582626399728538
Repeatition 1 Epoch 282 / 500 
 - time: 1.2468557357788086 - sq_loss: 6.2104522839945275e-06 - tot_loss: 0.09665709804590605 - acc: 0.9868063112078346 - val_acc: 0.9575839837122497
Repeatition 1 Epoch 283 / 500 
 - time: 1.2175498008728027 - sq_loss: 6.189643954712665e-06 - tot_loss: 0.0910527321144663 - acc: 0.9868063112078346 - val_acc: 0.9575839837122497
Repeatition 1 Epoch 284 / 500 
 - time: 1.247359275817871 - sq_loss: 6.17216392129194e-06 - tot_loss:

Repeatition 1 Epoch 325 / 500 
 - time: 1.2339537143707275 - sq_loss: 5.598942607321078e-06 - tot_loss: 0.10038453533907088 - acc: 0.9883025027203483 - val_acc: 0.9592806243637597
Repeatition 1 Epoch 326 / 500 
 - time: 1.246248483657837 - sq_loss: 5.584806785918772e-06 - tot_loss: 0.08935485770434326 - acc: 0.9883025027203483 - val_acc: 0.9586019681031558
Repeatition 1 Epoch 327 / 500 
 - time: 1.23966383934021 - sq_loss: 5.574353053816594e-06 - tot_loss: 0.09533509195371082 - acc: 0.9883025027203483 - val_acc: 0.9586019681031558
Repeatition 1 Epoch 328 / 500 
 - time: 1.260188102722168 - sq_loss: 5.56059785594698e-06 - tot_loss: 0.08794344862558745 - acc: 0.9883025027203483 - val_acc: 0.9586019681031558
Repeatition 1 Epoch 329 / 500 
 - time: 1.253178358078003 - sq_loss: 5.54662392460159e-06 - tot_loss: 0.09214123323705437 - acc: 0.9883025027203483 - val_acc: 0.9586019681031558
Repeatition 1 Epoch 330 / 500 
 - time: 1.2482500076293945 - sq_loss: 5.533604962693062e-06 - tot_loss: 0.0

Repeatition 1 Epoch 371 / 500 
 - time: 1.220482587814331 - sq_loss: 5.083182259113528e-06 - tot_loss: 0.09033361655448147 - acc: 0.9888465723612623 - val_acc: 0.9599592806243638
Repeatition 1 Epoch 372 / 500 
 - time: 1.2245738506317139 - sq_loss: 5.072346993983956e-06 - tot_loss: 0.09057524605503176 - acc: 0.9889825897714908 - val_acc: 0.9592806243637597
Repeatition 1 Epoch 373 / 500 
 - time: 1.2200098037719727 - sq_loss: 5.05837533637532e-06 - tot_loss: 0.09224728410968375 - acc: 0.9889825897714908 - val_acc: 0.9592806243637597
Repeatition 1 Epoch 374 / 500 
 - time: 1.2215631008148193 - sq_loss: 5.047205831942847e-06 - tot_loss: 0.08740881990585514 - acc: 0.9888465723612623 - val_acc: 0.9592806243637597
Repeatition 1 Epoch 375 / 500 
 - time: 1.2345025539398193 - sq_loss: 5.037221853854135e-06 - tot_loss: 0.09151928418997102 - acc: 0.9889825897714908 - val_acc: 0.9592806243637597
Repeatition 1 Epoch 376 / 500 
 - time: 1.2281067371368408 - sq_loss: 5.026991402701242e-06 - tot_loss

Repeatition 1 Epoch 417 / 500 
 - time: 1.2483255863189697 - sq_loss: 4.664160314860055e-06 - tot_loss: 0.087587759158966 - acc: 0.9892546245919478 - val_acc: 0.9630132337970818
Repeatition 1 Epoch 418 / 500 
 - time: 1.2485108375549316 - sq_loss: 4.65683206130052e-06 - tot_loss: 0.08969962311567592 - acc: 0.9893906420021763 - val_acc: 0.9619952494061758
Repeatition 1 Epoch 419 / 500 
 - time: 1.250889778137207 - sq_loss: 4.64841377834091e-06 - tot_loss: 0.0946375720100825 - acc: 0.9892546245919478 - val_acc: 0.9623345775364778
Repeatition 1 Epoch 420 / 500 
 - time: 1.2450919151306152 - sq_loss: 4.640006409317721e-06 - tot_loss: 0.09268996571526245 - acc: 0.9893906420021763 - val_acc: 0.9616559212758737
Repeatition 1 Epoch 421 / 500 
 - time: 1.2522318363189697 - sq_loss: 4.632493528333725e-06 - tot_loss: 0.08558440248058297 - acc: 0.9893906420021763 - val_acc: 0.9623345775364778
Repeatition 1 Epoch 422 / 500 
 - time: 1.2492949962615967 - sq_loss: 4.625277142622508e-06 - tot_loss: 0.

Repeatition 1 Epoch 463 / 500 
 - time: 1.2803986072540283 - sq_loss: 4.32874412581441e-06 - tot_loss: 0.08926396515854051 - acc: 0.9899347116430903 - val_acc: 0.9616559212758737
Repeatition 1 Epoch 464 / 500 
 - time: 1.2794914245605469 - sq_loss: 4.318967512517702e-06 - tot_loss: 0.08852120544122855 - acc: 0.9897986942328618 - val_acc: 0.9613165931455717
Repeatition 1 Epoch 465 / 500 
 - time: 1.2456233501434326 - sq_loss: 4.313462795835221e-06 - tot_loss: 0.08763441902404168 - acc: 0.9899347116430903 - val_acc: 0.9616559212758737
Repeatition 1 Epoch 466 / 500 
 - time: 1.2475261688232422 - sq_loss: 4.304815320210764e-06 - tot_loss: 0.08893460211664106 - acc: 0.9899347116430903 - val_acc: 0.9616559212758737
Repeatition 1 Epoch 467 / 500 
 - time: 1.2583916187286377 - sq_loss: 4.296471615816699e-06 - tot_loss: 0.09073902316829141 - acc: 0.9899347116430903 - val_acc: 0.9616559212758737
Repeatition 1 Epoch 468 / 500 
 - time: 1.2544090747833252 - sq_loss: 4.289507614885224e-06 - tot_los